# Winds Measured at Ship Compared to Nearby Buoy

## Example for Ocean Observatories Initiative Irminger Sea Cruise and Buoy turn from Deployment 10 to 11

This notebook compares the Ocean Observatories Initiative Irminger 11 Deployment buoy and ship wind sensor comparison. During this analysis, you will compare six different instruments attached to buoys and two sensors attached to the ship. 

* METBK 1 deployment 10 (Instrument: Gill 2-axis - new firmware)
* METBK 1 deployment 11 (Instrument: Gill 2-axis - old firmware)
* METBK 2 deployment 11 (Instrument: RM Young)
* FDCHP  deployment 11 (Instrument: Gill R3 3-axis)
* Ship Port and Starboard Sensors (Vaisala WXT520)


520)



### Preconditions:

- Imput files that for ship MET data
- 


### Imput files:
- Ship MET.csv files
- Ship MET.txt files

### Output

- Plots used for inspections along with well writen workflow
- Hourly plot of Ship and Buoy sensors with standard deviation for comparisons

In [ ]:
# Import the necessary packages
import os
import numpy as np
import pandas as pd
import xarray as xr
import csv
from os import path
# Import typing for function docstrings
import typing
from typing import Union, Tuple, List
import numpy.typing as npt

### Load the ship met data
I wrote a module of functions (the underway.py file) to parse and load the underway data. So we'll import those functions and use them to get the data into a nice dataset

In [ ]:
from underway import *

In [ ]:
# Enter in the directory you like the file to be located.
ship_dir = "/Users/aaron.wickware/Documents/PEP2024/Irminger_Sea-11/Irminger_Sea-11"
ship_met_files = sorted(["/".join((ship_dir, x)) for x in os.listdir(ship_dir) if x.endswith('.csv')])
met_headers = ["/".join((ship_dir, x)) for x in os.listdir(ship_dir) if 'MET_X' in x]
par_header = "/".join((ship_dir, 'MET_PAR.txt'))
rad_header = "/".join((ship_dir, 'MET_RAD.txt'))

# Parse the underway data
underway = parse_ship_met_data(ship_met_files, ATTRS, met_headers=met_headers, par_header=par_header, rad_header=rad_header)
underway

In [ ]:
met_headers

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
colors = ['#377eb8','#ff7f00','#FFC20A','#00ffff','#CC79A7','#4B0092']

In [ ]:
# Plot a timeseries of the ship observations
fig, ax = plt.subplots(figsize=(12, 8))

ax.plot(underway["time"], underway["wind_speed_starboard"], marker=".", linestyle="", label="Starboard Wind Sensor (Vaisala WXT520)", color=colors[2])
ax.plot(underway["time"], underway["wind_speed_port"], marker=".", linestyle="", label="Port Wind Sensor (Vaisala WXT520)",color=colors[1])
ax.legend()
ax.set_title('Irminger 11 Ship Wind Observations', fontsize=16)
ax.set_ylabel('Wind Speed (m/s)')
ax.grid()

fig.autofmt_xdate()

## Irminger Buoy Data

In [ ]:
from erddapy import ERDDAP

Next, we want to connect to the ERDDAP server which has the telemetered data. Since the data is not yet being ingested into the public-facing OOI Data Explorer, we're going to access the data via the OMS++ system. OMS is the internal operators dashboard which contains much of the same data. We will search for and download the data on the OMS++ erddap server.

In [ ]:
oms = ERDDAP(
    server="https://cgsn-dashboard.whoi.edu/erddap/",
    protocol="tabledap",
)

Now that we're connected to the ERDDAP server, we want to search for the available datasets

In [ ]:
search_url = oms.get_search_url(response="csv")
search = pd.read_csv(search_url)
datasets = search["Dataset ID"]
datasets

Wow. That is a lot of datasets that are availabe for download/access via OMS++. We need to figure out which one is for our data. We'll look for the array "GI01SUMO" and the instrument "METBK" to be in the dataset name/id

In [ ]:
for d in datasets:
    if "GI01SUMO" in d.upper() and "METBK" in d.upper():
        print(d)

We also want to find the FDCHP instrument (Direct Flux Covariance) which also measures wind speed using a three-axis sonic anemometer

In [ ]:
for d in datasets:
    if "GI01SUMO" in d.upper() and "FDCHP" in d.upper():
        print(d)

So the Irminger buoy has two datasets: METBK 1 and METBK 2. Additionally, during mooring turns, the _new_ buoy is deployed before the _old_ buoy is recovered. This means that there is a period of time when two buoys, and thus four wind sensors, were in the water. Combined with the ship observations, this makes for six datasets for comparison. So we want to query for the following Datasets:
* GI01SUMO-BUOY-METBK-01-1: Deployment 10 (D0010) & Deployment 11 (D0011)
* GI01SUMO-BUOY-METBK-02-1: Deployment 10 (D0010) & Deployment 11 (D0011)
* GI01SUMO-BUOY-FDCHP-01-1: Deployment 10 (D0010) & Deployment 11 (D0011)

If we just query the OMS++ ERDDAP server for a given dataset id (e.g. GI01SUMO-BUOY-METBK-01-1) that will provide us _all_ of the data for that dataset available from OMS++. However, we only want data for a given deployment number and for the times which overlap the ship observations. We can pass in constraints to the ERDDAP server to limit what data we get back. The constraints we want to use are:
* deploy_id=
* time>=
* time<=
where deploy_id is a given deployment (e.g. D0010) the times are the bounding start and end times for the data, passed in using the format "YYYY-mm-ddTHH:MM:SS".

First, get the time limits of the ship observations:

In [ ]:
startTime = underway.time.min()
stopTime = underway.time.max()

Next, lets get the METBK-01-1 dataset for deployment 10:

In [ ]:
# Set the constraints
oms.dataset_id = "GI01SUMO-BUOY-FDCHP-01-1"
oms.constraints = {
    "deploy_id=": "D0011",
    "time>=": '2024-06-02T00:00:01',
    "time<=": '2024-06-12T00:00:01'
}

# Convert the data to a pandas dataframe indexed by time
d11_fdchp1 = oms.to_pandas(index_col='time (UTC)', parse_dates=True)
d11_fdchp1

In [ ]:
# Now lets get the rest of the datasets
oms.dataset_id = "GI01SUMO-BUOY-METBK-01-1"
oms.constraints = {
    "deploy_id=": "D0010",
    "time>=": '2024-06-10T00:00:00',
    "time<=": '2024-06-11T07:00:00'
}


d10_metbk1 = oms.to_pandas(index_col='time (UTC)', parse_dates=True)
d10_metbk1

In [ ]:
oms.dataset_id = "GI01SUMO-BUOY-METBK-01-1"
oms.constraints = {
    "deploy_id=": "D0011",
    "time>=": '2024-06-10T00:00:00',
    "time<=": '2024-06-11T07:00:00'
}

d11_metbk1 = oms.to_pandas(index_col='time (UTC)', parse_dates=True)
d11_metbk1

## 

In [ ]:
oms.dataset_id = "GI01SUMO-BUOY-METBK-02-1"
oms.constraints = {
    "deploy_id=": "D0011",
    "time>=": '2024-06-10T00:00:00',
    "time<=": '2024-06-11T07:00:00'
}

d11_metbk2 = oms.to_pandas(index_col='time (UTC)', parse_dates=True)
d11_metbk2

In [ ]:
oms.dataset_id = "GI01SUMO-BUOY-FDCHP-01-1"
oms.constraints = {
    "deploy_id=": "D0011",
    "time>=": '2024-06-10T00:00:00',
    "time<=": '2024-06-11T07:00:00'
}

d11_fdchp1 = oms.to_pandas(index_col='time (UTC)', parse_dates=True)
d11_fdchp1

Before we plot we need to calculate the wind speed from our METBK1 and METBK2 eastward and northward vectors.

$$
\|{U}\| = \sqrt{u^{2} + v^{2}}
$$

Remember where $\|{U}\|$ is the magnitude of the wind speed, $u$ is the eastward vector wind speed, and $v$ is the northward vector wind speed. So we can go ahead and calculate ttha

Note: FDCHP will provide wind speed instead of the vectors so we will not need to calculatitt:

In [ ]:
wspd_METBK1_D10 = np.sqrt(d10_metbk1["northward_wind_velocity (m s-1)"]**2 + d10_metbk1["eastward_wind_velocity (m s-1)"]**2)
wspd_METBK1_D10

In [ ]:
wspd_METBK1_D11 = np.sqrt(d11_metbk1["northward_wind_velocity (m s-1)"]**2 + d11_metbk1["eastward_wind_velocity (m s-1)"]**2)
wspd_METBK1_D11

In [ ]:
wspd_METBK2_D11 = np.sqrt(d11_metbk2["northward_wind_velocity (m s-1)"]**2 + d11_metbk2["eastward_wind_velocity (m s-1)"]**2)
wspd_METBK2_D11

In [ ]:
# Add the calculated results
d10_metbk1['wspd_METBK1_D10'] = wspd_METBK1_D10
d11_metbk1['wspd_METBK1_D11'] = wspd_METBK1_D11
d11_metbk2['wspd_METBK2_D11'] = wspd_METBK2_D11

In [ ]:
# Plot some comparison time series figures here (Use same colore for the whole notebook)
fig, ax = plt.subplots(figsize=(12, 8))

ax.plot(d10_metbk1.index, d10_metbk1["wspd_METBK1_D10"], marker=".", linestyle="", label="METBK1 d10 Gill 2-axis",color=colors[0])
ax.plot(d11_metbk1.index, d11_metbk1["wspd_METBK1_D11"], marker=".", linestyle="", label="METBK1 d11 Gill 2-axis",color=colors[3])

ax.legend()
ax.set_title('Irminger 10 and 11 Buoy Wind Observations (METBK1)', fontsize=16)
ax.set_ylabel('Wind Speed (m/s)')
ax.grid()

fig.autofmt_xdate()

In [ ]:
#examine the dataframe
d11_fdchp1

In [ ]:
# Plot with all of our buoy data
fig, ax = plt.subplots(figsize=(12, 8))

ax.plot(d10_metbk1.index, d10_metbk1["wspd_METBK1_D10"], marker=".", linestyle="", label="METBK1 d10 Gill 2-axis",color=colors[0])
ax.plot(d11_metbk1.index, d11_metbk1["wspd_METBK1_D11"], marker=".", linestyle="", label="METBK1 d11 Gill 2-axis", color=colors[3], alpha=0.6,)
ax.plot(d11_fdchp1.index, d11_fdchp1["wind_speed"], marker=".", linestyle="", label="FDCHP d11 Gill R3 3-axis", color=colors[4])
ax.plot(d11_metbk2.index, d11_metbk2["wspd_METBK2_D11"], marker=".", linestyle="", label="METBK2 d11 RM Young", color=colors[5], alpha = 0.3)

ax.legend()
ax.set_title('Irminger 10 and 11 Buoy Wind Observations', fontsize=16)
ax.set_ylabel('Wind Speed (m/s)')
ax.grid()

fig.autofmt_xdate()

---
### Adjust the data
One complicating factor in making an comparisons between the ship and buoy wind data is that the ship data is measured from the ship mast which, on the Armstrong, is at 17.9 m height. In comparison, the buoy MET wind sensors are mounted on the mast of the buoy. The buoy deck sits 45 cm above the water line, while the wind sensors and FDCHP are mounted 540 cm above the buoy deck. So we need to correct for the height differences between the sensors on the Armstrong and the buoys. We do this by defining a function based on the COARE (Coupled Ocean-Atmosphere Response Experiment) 3.5 flux algorithms.

In [ ]:
# We first need to calculate the 10-meter wind speeds U10 and the friction-velocity u* (ustar)
def dragNC35(z: Union[int, float], U: npt.NDArray[float]) -> Tuple[npt.NDArray[float], npt.NDArray[float]]:
    """
    Calculate 10 meter winds and u* winds using COARE 3.5 algorithm
    
    Parameters
    ----------
    z: float
        The height of the wind sensor in meters
    U: float
        The measured wind speed by the wind sensor

    Returns
    -------
    U10: float
        The 10 meter winds
    Ustar: float
        The friction velocity
    """

    # Define constants
    wstr = 0.2/1.2
    ug = 0.2
    von = 0.4
    visc = 1.45E-5
    umax = 19
    a1 = 0.0017
    a2 = -0.005
    charnold = 0.011
    rnn = 1/9

    # Run the COARE 3.5 algorithm
    # Initialize values
    ut = np.sqrt(U*U + ug*ug)
    us = 0.035*ut
    charn=a1*ut + a2
    mask = (charn > umax)
    charn[mask] = a1*umax+a2
    # Iterate 10 times, updating with each successive pass
    i=0
    while i<10:
        z0 = visc/us*rnn + charn*us*us/9.8
        us = ut*von/np.log(z/z0)
        u10 = ut + us/von*np.log(10/z)
        # Update the charn variable
        mask = (u10 > umax)
        charn[mask] = a1*umax + a2
        charn[~mask] = a1*u10[~mask] + a2
        # Calculate ustar
        ustar = us
        # Calculate 10m winds
        U10 = ut + us/von*np.log(10/z)
        # Update counter
        i = i + 1

    return U10, ustar

# Next, we write a function which uses the u* values to adjust the heights
def adjust_height(wspd: npt.NDArray[float], ustar: npt.NDArray[float], z: Union[int, float], z0: Union[int, float]) -> npt.NDArray[float]:
    """
    Adjust the height of a wind sensor from height z to height z0
    
    Parameters
    ----------
    wspd: numpy.array[float]
        An array of observed wind speeds at height z
    ustar: numpy.array[float]
        An array of ustar values for the associated wspd at height
        z
    z: int | float
        Height at which the winds were observed in meters
    z0: int | float
        Height to adjust the observed wind speeds to in meters

    Returns
    -------
    adjusted: numpy.array[float]
        An array of the observed wind speeds adjusted from height z to height z0
    """

    correction = (ustar/0.4)*np.log(z0/z)
    adjusted = wspd + correction

    return adjusted

In [ ]:
# Calculate the U10 and ustar 
U10, ustar = dragNC35(17.9, underway["wind_speed_starboard"].values) # z = 17.9

# Add the ustar to the underway data
underway["friction_velocity_starboard"] = (["time"], ustar)
underway["friction_velocity_starboard"].attrs = {
    'standard_name': 'friction_velocity',
    'long_name': 'Friction Velocity - Starboard',
    'units': 'm s-1',
    'comment': ('Friction velocity is a reference wind velocity that relates the Reynold\'s stress with the density. It is '
                'applied to motion near the ground where the shearing stress is assumed to be independent of height and '
                'proportional to the square of the mean velocity.')
}

# Repeat for the port sensor
U10, ustar = dragNC35(17.9, underway["wind_speed_port"].values)

# Add the ustar to the underway data
underway["friction_velocity_port"] = (["time"], ustar)
underway["friction_velocity_port"].attrs = {
    'standard_name': 'friction_velocity',
    'long_name': 'Friction Velocity - Port',
    'units': 'm s-1',
    'comment': ('Friction velocity is a reference wind velocity that relates the Reynold\'s stress with the density. It is '
                'applied to motion near the ground where the shearing stress is assumed to be independent of height and '
                'proportional to the square of the mean velocity.')
}

In [ ]:
# Calculate the adjusted wind speed
wspd_starboard = underway["wind_speed_starboard"]
ustar_starboard = underway["friction_velocity_starboard"]
underway["adj_wind_speed_starboard"] = adjust_height(wspd_starboard, ustar, 17.9, 5.85)
underway["adj_wind_speed_starboard"]

# Repeat for the port sensor
wspd_port = underway["wind_speed_port"]
ustar_port = underway["friction_velocity_port"]
underway["adj_wind_speed_port"] = adjust_height(wspd_port, ustar, 17.9, 5.85)
underway["adj_wind_speed_port"]

In [ ]:
os.getcwd()

In [ ]:
!pip install h5netcdf

Now we have data that are directly comparable to each other. I would recommend saving your different datasets as new datasets locally so you don't have to go through the different data access steps. Now we are ready to begin analysis!

In [ ]:
# We are going to slice our 
underway.sel(time = slice('2024-06-10 00:00:00','2024-06-11 07:00:00'))

In [ ]:
underway_june10_june11 = underway.sel(time = slice('2024-06-10 00:00:00','2024-06-11 07:00:00'))

In [ ]:
# Examine our adjusted data and plot
underway_june10_june11

In [ ]:
# Now let's plot our time series
fig, ax = plt.subplots(figsize=(12, 8))

ax.plot(underway_june10_june11["time"], underway_june10_june11["adj_wind_speed_starboard"], marker=".", linestyle="", label="Adjusted Starboard Wind Sensor (Vaisala WXT520)",color=colors[2])
ax.plot(underway_june10_june11["time"], underway_june10_june11["adj_wind_speed_port"], marker=".", linestyle="", label="Adjusted Port Wind Sensor (Vaisala WXT520)",color=colors[1])

ax.legend()
ax.set_title('Irminger 11 Ship Wind Observations', fontsize=16)
ax.set_ylabel('Wind Speed (m/s)')
ax.grid()

fig.autofmt_xdate()

In [ ]:
# Plot a timeseries of the ship observations
fig, ax = plt.subplots(figsize=(12, 8))

ax.plot(underway_june10_june11["time"], underway_june10_june11["adj_wind_speed_starboard"], marker=".", linestyle="", label="Starboard Wind Sensor (Vaisala WXT520)",color=colors[2])
ax.plot(underway_june10_june11["time"], underway_june10_june11["adj_wind_speed_port"], marker=".", linestyle="", label="Port Wind Sensor (Vaisala WXT520)",color=colors[1])
ax.plot(d10_metbk1.index, d10_metbk1["wspd_METBK1_D10"], marker=".", linestyle="", label="METBK1 D10 (Gill 2-axis)", color=colors[0])
ax.plot(d11_metbk1.index, d11_metbk1["wspd_METBK1_D11"], marker=".", linestyle="", label="METBK1 D11 (Instrument: Gill 2-axis)", color=colors[3])
ax.plot(d11_fdchp1.index, d11_fdchp1["wind_speed"], marker=".", linestyle="", label="FDCHP D11 (Gill R3 3-axis)", color=colors[4])
ax.plot(d11_metbk2.index, d11_metbk2["wspd_METBK2_D11"], marker=".", linestyle="", label="METBK2 D11 (RM Young)", color=colors[5], alpha = 0.6)


ax.legend()
ax.set_title('Irminger 11 Ship Wind Observations', fontsize=16)
ax.set_ylabel('Wind Speed (m/s)')
ax.grid()

fig.autofmt_xdate()

Now we have a time series of all of data, how does it look? 

If your graph looks good then your done, but if you have some loose points we can try and take the mean of our data to try and clean up the graph and declutter a little bit.

In [ ]:
d11_metbk1

Note: if there are floats you will not be able resample the data or run the mean or std. functions, to fix this we want to drop all the qualitative columns in our data set: *Deployment Identification*.

FDCHP will also have *Featured Deployment Name* so we will drop that one aswell.

In [ ]:
d11_fdchp = d11_fdchp1.drop(columns='deploy_id (1)')
d11_fdchp

In [ ]:
d11_fdchp = d11_fdchp.drop(columns='feature_type_instance')

In [ ]:
underway_june10_june11

In [ ]:
# Assign mean variable, assign std. variable.
d11_metbk1_mean = d11_metbk1.drop(columns='deploy_id (1)').resample('1H').mean()
d10_metbk1_mean = d10_metbk1.drop(columns='deploy_id (1)').resample('1H').mean()
d11_metbk2_mean = d11_metbk2.drop(columns='deploy_id (1)').resample('1H').mean()

In [ ]:
d11_metbk1_std = d11_metbk1.drop(columns='deploy_id (1)').resample('1H').std()
d10_metbk1_std = d10_metbk1.drop(columns='deploy_id (1)').resample('1H').std()
d11_metbk2_std = d11_metbk2.drop(columns='deploy_id (1)').resample('1H').std()

In [ ]:
d11_fdchp.columns
d11_fdchp_mean=d11_fdchp.drop(columns='dcl_date_time_string (1)').resample('1H').mean()

In [ ]:
d11_fdchp_std=d11_fdchp.drop(columns='dcl_date_time_string (1)').resample('1H').std()

In [ ]:
underway_june10_june11_mean=underway_june10_june11.resample(time = '1H').mean()

In [ ]:
underway_june10_june11_std=underway_june10_june11.resample(time = '1H').std()

In [ ]:
underway_june10_june11_mean["time"]

In [ ]:
## Plot a timeseries of the ship observations averages (Red/Green colorblind friendly)
fig, ax = plt.subplots(figsize=(12, 8))


ax.plot(underway_june10_june11_mean["time"], underway_june10_june11_mean["adj_wind_speed_starboard"], marker=".", linestyle="", label="Starboard Wind Sensor (Vaisala WXT520)", color=colors[2])
ax.plot(underway_june10_june11_mean["time"], underway_june10_june11_mean["adj_wind_speed_port"], marker=".", linestyle="", label="Port Wind Sensor (Vaisala WXT520)",color=colors[1])
ax.plot(d10_metbk1_mean.index, d10_metbk1_mean["wspd_METBK1_D10"], marker=".", linestyle="", label="METBK1 D10 (Gill 2-axis)",color=colors[0])
ax.plot(d11_metbk1_mean.index, d11_metbk1_mean["wspd_METBK1_D11"], marker=".", linestyle="", label="METBK1 D11 (Gill 2-axis)", color=colors[3])
ax.plot(d11_fdchp_mean.index, d11_fdchp_mean["wind_speed"], marker=".", linestyle="", label="FDCHP D11 (Gill R3 3-axis)", color=colors[4])
ax.plot(d11_metbk2_mean.index, d11_metbk2_mean["wspd_METBK2_D11"], marker=".", linestyle="", label="METBK2 D11 (RM Young)", color=colors[5])

# Plot the Error
ax.errorbar(x=d11_metbk1_mean.index,y=d11_metbk1_mean["wspd_METBK1_D11"] , yerr=d11_metbk1_std["wspd_METBK1_D11"],color=colors[3])
ax.errorbar(x=d10_metbk1_mean.index,y=d10_metbk1_mean["wspd_METBK1_D10"] , yerr=d10_metbk1_std["wspd_METBK1_D10"], color=colors[0])
ax.errorbar(x=d11_metbk2_mean.index,y=d11_metbk2_mean["wspd_METBK2_D11"] , yerr=d11_metbk2_std["wspd_METBK2_D11"], color=colors[5])
ax.errorbar(x=d11_fdchp_mean.index,y=d11_fdchp_mean["wind_speed"] , yerr=d11_fdchp_std["wind_speed"], color=colors[4])
ax.errorbar(x=underway_june10_june11_mean['time'].data, y=underway_june10_june11_mean["adj_wind_speed_starboard"].data , yerr=underway_june10_june11_std["adj_wind_speed_starboard"].data, color=colors[2])
ax.errorbar(x=underway_june10_june11_mean['time'].data, y=underway_june10_june11_mean["adj_wind_speed_port"].data , yerr=underway_june10_june11_std["adj_wind_speed_port"].data, color=colors[1])

ax.legend()
ax.set_title('Irminger 11 Wind Observations Averages', fontsize=16)
ax.set_ylabel('Wind Speed (m/s)')
ax.grid()

fig.autofmt_xdate()


In [ ]:
file_path = path.abspath("/Users/aaron.wickware/Documents/PEP2024/Ship_buoy_comparisons.csv")

len(file_path)

In [ ]:
underway_adj_wspd = underway_june10_june11_mean[['adj_wind_speed_starboard', 'adj_wind_speed_port']].to_dataframe()

In [ ]:
underway_adj_wspd.to_csv(file_path)

In [ ]:
#--------------------------

In [ ]:
#underway_june10_june11 = underway_june10_june11.resample(time = '1min').mean()

In [ ]:
underway_june10_june11

In [ ]:
underway_june10_june11 = underway_june10_june11[['adj_wind_speed_starboard', 'adj_wind_speed_port']].to_dataframe()

In [ ]:
file_path = path.abspath("/Users/aaron.wickware/Documents/PEP2024/underway_june10_june11.csv")

In [ ]:
underway_june10_june11_1min = underway_june10_june11[['adj_wind_speed_starboard', 'adj_wind_speed_port']].to_csv(file_path)

In [ ]:
underway_june10_june11_1min